In [ ]:
import cdsapi
import geopandas as gpd
from time import sleep

---

In [ ]:
client = cdsapi.Client()

In [ ]:
path_to_shapefiles = "/Users/cooper/Desktop/CAMELS-CH/data/catchment_delineations/CAMELS_CH_catchments.shp"

gdf = gpd.read_file(path_to_shapefiles).to_crs(epsg=4326)

In [ ]:
def get_bounding_box(gdf):
    """
    Get the bounding box coordinates for a GeoDataFrame.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame containing geometries

    Returns:
        tuple: Bounding box coordinates in format (North, West, South, East)

    Example:
        >>> bbox = get_bounding_box(gdf)
        >>> print(bbox)
        (47.808, 5.956, 45.817, 10.492)
    """
    total_bounds = gdf.total_bounds
    # total_bounds returns (minx, miny, maxx, maxy)
    # Convert to (North, West, South, East) format
    west = total_bounds[0]
    south = total_bounds[1]
    east = total_bounds[2]
    north = total_bounds[3]

    return (north, west, south, east)


bbox = get_bounding_box(gdf)
bbox

In [ ]:
north, west, south, east = bbox

for year in range(1992, 2020):
    temp_request = {
        "format": "netcdf",
        "download_format": "zip",
        "variable": "2m_temperature",
        "year": [str(year)],
        "month": [f"{i:02d}" for i in range(1, 13)],
        "day": [f"{i:02d}" for i in range(1, 32)],
        "time": ["00:00", "06:00", "12:00", "18:00"],
        "area": [north, west, south, east],
    }

    precip_request = {
        "format": "netcdf",
        "download_format": "zip",
        "variable": "total_precipitation",
        "year": [str(year)],
        "month": [f"{i:02d}" for i in range(1, 13)],
        "day": [f"{i:02d}" for i in range(1, 32)],
        "time": ["00:00"],
        "area": [north, west, south, east],
    }

    client.retrieve(
        "reanalysis-era5-land",
        temp_request,
        f"/Users/cooper/Desktop/CAMELS-CH/data/ERA5_land_1981-2020/2m_temperature_{year}.nc",
    )

    client.retrieve(
        "reanalysis-era5-land",
        precip_request,
        f"/Users/cooper/Desktop/CAMELS-CH/data/ERA5_land_1981-2020/total_precipitation_{year}.nc",
    )

    sleep(5)